## 2. Q-Learning
Last week, we learned the basics of policy gradient algorithms through REINFORCE and A2C implementation. The working principle of such algorithms mostly aimed at optimizing the policy directly, so the action probability distribution ensures the maximum expected reward. This week, we are going to analyze ***value-based*** approach starting with Q-Learning algorithm.

### 2.1 What is Q-Learning? 
Q-learning is a reinforcement learning algorithm that is used to learn an optimal action-selection strategy for any given decision-making problem. It is based on the idea of using a Q-value function, which is a mathematical representation of the expected return of taking a particular action in a particular state.

The basic idea behind Q-learning is to start with an initial Q-value function, and then iteratively update it as the agent interacts with its environment. At each iteration, the agent observes the current state of the environment, selects an action based on its current Q-values (using an exploration-exploitation tradeoff), receives a reward from the environment, and transitions to a new state. The Q-values are then updated using the observed reward and the estimated future rewards that can be obtained by taking the best action in the next state.

Over time, as the agent continues to interact with the environment and update its Q-values, the Q-value function converges to the optimal Q-value function, which gives the accurate expected utility for each state-action pair.


In the case of path planning problem that we have shown in the previous notebook, we have an empty q-value table. However, we don't really know what is the q-value function for each of the state-action pair. What Q-learning (and DQN) does  is it try to approximate the Q-value for each state-action pair by doing an iterative update so that it converges to the actual state-action pair. 

**Why do we want to do this?**\
Finding the approximate Q-value function for each state-action pair that is close to its true (or optimal) value will enable us to extract the optimal action for any state we are in, hence allowing the robot to have an optimal trajectory to go from its current cell to the 'End' cell

<img src="img/q_value_01.png" width="400">


### 2.2 Algorithm

Shown below is the step-by-step instruction of performing this algorithm:

1. **Initiatizing Q-value space.** In order to record the data, model will have an associated Q-table containing all Q-values of the environment.
2. **Extracting action probabilities**. At the start of each episode, we need to retrieve the probabilities of all actions in that state.
3. **Taking action**. The action is taken based on **epsilon-greedy algorithm**. Either the agent will take a random action or choose the action that has the largest probability according to the probability distribution.
4. **Recording data**. We need to extract reward, next state after taking a sample action.
5. **Updating Q-table**. After getting the loss, we can update our table by adding difference multiplied by the learning rate.



**What is epsilon-greedy policy?**\
The Q-Learning is based on the **epsilon-greedy policy** principle: either the agent will explore the environment or the agent will exploit the environment by choosing the optimal action (with the highest reward). Shown below is the formula for epsilon-greedy policy:

<img src="img/epsilon-greedy-formula.png" width=400>

If the agent decided to choose to exploit the environment by picking the optimal state, the relationship between the state value and Q-value functions:

$$
V(s_t) = max_a Q(s_t, a)
$$

Plugging this into the previously derived equation, we get the Bellman equation for the Q-Learning.

$$
Q^{\pi}(s_t, a_t) = r(s_t, a_t) + \gamma max_{a_{t+1}} Q(s_{t+1}, a_{t+1})
$$

In the descriptive terms, this equation tells that the of an action in a certain state is the **immediate reward** we get from taking the action and the **maximum expected reward** in the next state.

<!--![greedy_algo](https://miro.medium.com/max/375/0*rQ7hXKOPSxcR271w.gif)-->

In summary, what Q-learning is trying to do is to find the optimal Q-value function for each state. This is done by doing an iterative update until the Q-value function has converged. By finding the optimal Q-value function for each state, this allows us to extract the optimal policy that will guide our RL agent to attain the maximum or the highest reward.

## 3. Deep Q-Network
### 3.1 Motivation

Regular Q-Learning trains by taking series of actions and creating a map that connects each state-action pair to its corresponding Q-value iteratively. In the case of complex, large environments, such approach becomes computationally intense and inefficient.

One way of dealing with such problems is to use a **neural network** that would map input states to action, Q-value pairs. This is the basis of **deep Q-networks** (or **DQN** for short). Essentially, DQN uses a deep neural network as a function approximator to estimate the Q-values. The neural network takes the current state as input and outputs a Q-value for each possible action. 

<img src=https://cdn.analyticsvidhya.com/wp-content/uploads/2019/04/Screenshot-2019-04-16-at-5.46.01-PM.png width="800">


### 3.2 Algorithm

The functioning of DQN can be divided into the following steps:

1. Initializing behaviour and target neural networks
2. Choosing an action using the epsilon-greedy policy
3. Updating weights using Bellman equation

### Target and Behaviour neural networks

The main difference between the simple Q-Learning and DQN is the way of mapping Q-values. DQN replaces the iterative technique that we used before with a neural network, and, rather than mapping a state-action pair to a Q-value, it maps input states to action, Q-value pairs.

In addition, DQN uses **2 neural networks** with the same architecture, called the behaviour network and the target network. The behaviour network is the neural network that is used to approximate the Q-values during training, and its weights are updated at each iteration of the algorithm. The target network, on the other hand, is a copy of the online network that is used to calculate the target Q-values during training.

The use of two separate networks and the "copy-and-paste" method are important because it helps to stabilize the learning process. In standard Q-learning, the Q-values are updated based on the same estimates that are being used to select actions, which can cause the estimates to become unstable or even diverge. By using a separate target network to calculate the target Q-values, the DQN algorithm is able to prevent these issues and improve the stability of learning.

The weights of the target network are updated less frequently than the online network, typically after a fixed number of iterations or after a certain number of experiences have been stored in the replay buffer. This delay in updating the target network helps to keep the target values consistent over time and improves the convergence of the algorithm.


<img src=https://miro.medium.com/max/875/0*9qs-EEw3iReB72Lf width="500">

From the structural perspective, each neural network receives a series of state inputs and outputs Q-values for each action. As the typical neural network we have learned before, the input and output is be connected by a series of hidden layers.

### Choosing an action

As in Q-Learning, the action in DQN is chosen according to the same **episilon-greedy** policy.

In DQN case, the neural network takes states as an input and produces Q-values for each action meaning that the action (output node) with the **largest Q-value** will be chosen had the algorithm choose to exploit the environment. If the algorithm decided to explore the environment, the agent will take a random action from a set of possible actions

### Updating weights

After the action is chosen, we need to update both networks according to our previously learned Bellman equation. In short, behaviour neural network samples and trains on a batch of past episodes every few steps. The weights of main network is then coppied to the target network every N steps. This is called **experience replay**.

**Experience replay** involves storing and replaying game states that RL algorithm is able to learn from. DQN uses it to learn on small batches to avoi skewing dsitrbution of the dataset. It is also not essential to train every step which allows us to control the network training speed.

#### Bellman equation

<img src=https://miro.medium.com/max/875/0*hVd8wqmFIEKQqGm9 width="500">

Similar to Q-Learning, DQN will use Bellmann equation to update its weigths. According to the equation, the target value takes the following form.

$$
Q^{\pi}(s_t, a_t) = r(s_t, a_t) + \gamma max_{a_{t+1}} Q(s_{t+1}, a_{t+1})
$$

It is important to note that the **target network** is used to calculate this target. Such output is then passed to the **main network** as a target Q-value that is then being used to train the **main network**.
